In [ ]:
#! python --version

#import sys
#!{sys.executable} -m pip install sqlparse
#!{sys.executable} -m pip install transformers
#!{sys.executable} -m pip install accelerate
#!{sys.executable} -m pip install jupyter ipywidgets

In [23]:
import sqlparse
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TRANSFORMERS_CACHE


In [24]:
print(torch.cuda.is_available())
torch.cuda.empty_cache()

available_memory = torch.cuda.get_device_properties(0).total_memory
print(available_memory)

print(TRANSFORMERS_CACHE)

True
8589410304
C:\Users\zly20\.cache\huggingface\hub


# DeepSeek-R1-Distill-Qwen-1.5B Presentation Demo

In [25]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
cache_dir = "D:/Data File/transformers.cache"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

if available_memory > 4e9:
    # if you have atleast 4GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
        cache_dir=cache_dir
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
        cache_dir=cache_dir
    )

Some parameters are on the meta device because they were offloaded to the cpu.


# STEP 1: No Prompt

In [26]:
user_input = """Generate a SQL query to answer [QUESTION]{question}[/QUESTION]"""

In [27]:
def generate_query(question):
    updated_input = user_input.format(question=question)
    inputs = tokenizer(updated_input, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return outputs[0].split("[/QUESTION]")[-1]

In [28]:
question = "List all the reviews which rated a business less than 1"
generated_sql = generate_query(question)
print(generated_sql)

 in the reviews table.
To answer the question, I need to write a SQL query that selects all records from the reviews table where the 'rating' column is less than 1. I should make sure to use the correct column names and syntax.
To write the query, I'll start by selecting the appropriate column, which is 'rating' in this case. Then, I'll use the 'WHERE' clause to filter the records where the 'rating' is less than 1. I'll also need to specify the table name, which is 'reviews', and ensure that the column names are correctly referenced. I'll make sure to use the correct SQL syntax for the 'where' clause, which is 'WHERE rating < 1'. This should accurately retrieve all reviews with a rating below 1.
</think>

To answer the question "List all the reviews which rated a business less than 1", you can write a SQL query that selects all records from the reviews table where the 'rating' column is less than 1. Here is the SQL query:

```sql
SELECT rating FROM reviews WHERE rating < 1;
```

This q

In [ ]:
"""
Corret Answer: 

SELECT r.text 
FROM review AS r 
WHERE r.rating < 1


CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);
"""

In [29]:
question = "list all the businesses which have a review by Niloofar"
generated_sql = generate_query(question)
print(generated_sql)

 in the database.

To answer this question, I need to:

1. Identify the table that contains the reviews.
2. Determine the columns in that table that represent the reviews.
3. Find the column name that indicates the business owner.
4. Use a SELECT statement to list all records where the business owner is Niloofar.
5. Order the results in ascending order based on the number of reviews.

I need to make sure that the SQL query is correct and that the columns are properly referenced.
To answer this question, I need to follow these steps:

1. **Identify the Table**: First, I need to determine which table contains the reviews. Typically, in a database, reviews might be stored under a table named something like "reviews" or "business_reviews". For this example, I'll assume the table is named "reviews".

2. **Determine the Columns**: Next, I should look at the columns in the "reviews" table to find the one that represents the business owner. This column might be named "business_owner" or "owner

In [ ]:
"""
Correct Answer: 

SELECT b.name
FROM review r
JOIN business b ON r.business_id = b.business_id
JOIN user u ON r.user_id = u.user_id
WHERE u.name = "Niloofar";


CREATE TABLE business (
  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business
  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier
  name VARCHAR(255) NOT NULL, -- Name of the business
  full_address VARCHAR(255) NOT NULL, -- Full street address
  city VARCHAR(255) NOT NULL, -- City of the business
  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate
  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate
  review_count BIGINT NOT NULL, -- Number of reviews the business has received
  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)
  rating FLOAT DEFAULT NULL, -- Average rating of the business
  state VARCHAR(255) DEFAULT NULL -- State or province
);

CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE user (
  uid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each user record
  user_id VARCHAR(255) NOT NULL UNIQUE, -- External user identifier
  name VARCHAR(255) NOT NULL -- Name of the user
);
"""

# STEP 2: Prompt with No Examples

In [30]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE business (
  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business
  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier
  name VARCHAR(255) NOT NULL, -- Name of the business
  full_address VARCHAR(255) NOT NULL, -- Full street address
  city VARCHAR(255) NOT NULL, -- City of the business
  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate
  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate
  review_count BIGINT NOT NULL, -- Number of reviews the business has received
  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)
  rating FLOAT DEFAULT NULL, -- Average rating of the business
  state VARCHAR(255) DEFAULT NULL -- State or province
);

CREATE TABLE category (
  id INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each category record
  business_id VARCHAR(255) NOT NULL, -- ID of the business this category belongs to
  category_name VARCHAR(255) NOT NULL, -- Name of the category
  FOREIGN KEY (business_id) REFERENCES business(business_id)
);

CREATE TABLE checkin (
  cid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each check-in record
  business_id VARCHAR(255) NOT NULL, -- ID of the business where the check-in occurred
  count INTEGER DEFAULT NULL, -- Number of check-ins
  day VARCHAR(12) DEFAULT NULL, -- Day of the week the check-in occurred
  FOREIGN KEY (business_id) REFERENCES business(business_id)
);

CREATE TABLE neighborhood (
  id INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each neighborhood record
  business_id VARCHAR(255) NOT NULL, -- ID of the business in the neighborhood
  neighborhood_name VARCHAR(255) NOT NULL, -- Name of the neighborhood
  FOREIGN KEY (business_id) REFERENCES business(business_id)
);

CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE tip (
  tip_id INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each tip
  business_id VARCHAR(255) NOT NULL, -- ID of the business the tip is about
  text LONGTEXT NOT NULL, -- Content of the tip
  user_id VARCHAR(255) NOT NULL, -- ID of the user who left the tip
  likes INTEGER NOT NULL, -- Number of likes the tip received
  year INTEGER DEFAULT NULL, -- Year the tip was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the tip was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE user (
  uid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each user record
  user_id VARCHAR(255) NOT NULL UNIQUE, -- External user identifier
  name VARCHAR(255) NOT NULL -- Name of the user
);

- review.business_id can be joined with business.business_id
- user.user_id can be joined with review.user_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [32]:
def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[1].split("[/SQL]")[0], reindent=True)

In [33]:
question = "List all the reviews which rated a business less than 1"
generated_sql = generate_query(question)
print(generated_sql)


SELECT r.*,
       b.is_open
FROM review r
LEFT JOIN business b ON r.business_id = b.business_id
WHERE r.rating < 1;


In [ ]:
"""
Corret Answer: 

SELECT r.text 
FROM review AS r 
WHERE r.rating < 1


CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);
"""

In [34]:
question = "list all the businesses which have a review by Niloofar"
generated_sql = generate_query(question)
print(generated_sql)


SELECT b.business_id
FROM business b
WHERE b.business_id IN
    (SELECT business.id
     FROM review r
     WHERE r.user_id = 'Niloofar'
       AND r.user_id = b.user_id);


In [35]:
"""
Correct Answer: 

SELECT b.name
FROM review r
JOIN business b ON r.business_id = b.business_id
JOIN user u ON r.user_id = u.user_id
WHERE u.name = "Niloofar";


CREATE TABLE business (
  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business
  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier
  name VARCHAR(255) NOT NULL, -- Name of the business
  full_address VARCHAR(255) NOT NULL, -- Full street address
  city VARCHAR(255) NOT NULL, -- City of the business
  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate
  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate
  review_count BIGINT NOT NULL, -- Number of reviews the business has received
  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)
  rating FLOAT DEFAULT NULL, -- Average rating of the business
  state VARCHAR(255) DEFAULT NULL -- State or province
);

CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE user (
  uid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each user record
  user_id VARCHAR(255) NOT NULL UNIQUE, -- External user identifier
  name VARCHAR(255) NOT NULL -- Name of the user
);
"""

'\nCorrect Answer: \n\nSELECT b.name\nFROM review r\nJOIN business b ON r.business_id = b.business_id\nJOIN user u ON r.user_id = u.user_id\nWHERE u.name = "Niloofar";\n\n\nCREATE TABLE business (\n  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business\n  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier\n  name VARCHAR(255) NOT NULL, -- Name of the business\n  full_address VARCHAR(255) NOT NULL, -- Full street address\n  city VARCHAR(255) NOT NULL, -- City of the business\n  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate\n  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate\n  review_count BIGINT NOT NULL, -- Number of reviews the business has received\n  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)\n  rating FLOAT DEFAULT NULL, -- Average rating of the business\n  state VARCHAR(255) DEFAULT NULL -- State or province\n);\n\nCREATE TABLE review (\n  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- 

# STEP 3: Prompt with Few-Shots:

In [36]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Pay close attention to the given database schema, note on how they can join together
- You do not need to generate your thought process but just the answer
- Your answer should end with '[/SQL]'

### Few-Shots Example:
  example input question 1: 'List all the reviews less than 3 stars'
  example answer 1: SELECT r.text FROM review AS r WHERE r.rating < 3

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE business (
  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business
  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier
  name VARCHAR(255) NOT NULL, -- Name of the business
  full_address VARCHAR(255) NOT NULL, -- Full street address
  city VARCHAR(255) NOT NULL, -- City of the business
  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate
  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate
  review_count BIGINT NOT NULL, -- Number of reviews the business has received
  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)
  rating FLOAT DEFAULT NULL, -- Average rating of the business
  state VARCHAR(255) DEFAULT NULL -- State or province
);

CREATE TABLE category (
  id INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each category record
  business_id VARCHAR(255) NOT NULL, -- ID of the business this category belongs to
  category_name VARCHAR(255) NOT NULL, -- Name of the category
  FOREIGN KEY (business_id) REFERENCES business(business_id)
);

CREATE TABLE checkin (
  cid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each check-in record
  business_id VARCHAR(255) NOT NULL, -- ID of the business where the check-in occurred
  count INTEGER DEFAULT NULL, -- Number of check-ins
  day VARCHAR(12) DEFAULT NULL, -- Day of the week the check-in occurred
  FOREIGN KEY (business_id) REFERENCES business(business_id)
);

CREATE TABLE neighborhood (
  id INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each neighborhood record
  business_id VARCHAR(255) NOT NULL, -- ID of the business in the neighborhood
  neighborhood_name VARCHAR(255) NOT NULL, -- Name of the neighborhood
  FOREIGN KEY (business_id) REFERENCES business(business_id)
);

CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE tip (
  tip_id INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each tip
  business_id VARCHAR(255) NOT NULL, -- ID of the business the tip is about
  text LONGTEXT NOT NULL, -- Content of the tip
  user_id VARCHAR(255) NOT NULL, -- ID of the user who left the tip
  likes INTEGER NOT NULL, -- Number of likes the tip received
  year INTEGER DEFAULT NULL, -- Year the tip was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the tip was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE user (
  uid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each user record
  user_id VARCHAR(255) NOT NULL UNIQUE, -- External user identifier
  name VARCHAR(255) NOT NULL -- Name of the user
);

- review.business_id can be joined with business.business_id
- user.user_id can be joined with review.user_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [37]:
def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[1].split("[/SQL]")[0], reindent=True)

In [38]:
question = "List all the reviews which rated a business less than 1"
generated_sql = generate_query(question)
print(generated_sql)


SELECT r.*
FROM review
WHERE r.rating < 1;


In [39]:
"""
Corret Answer: 

SELECT r.text 
FROM review AS r 
WHERE r.rating < 1


CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);
"""

'\nCorret Answer: \n\nSELECT r.text \nFROM review AS r \nWHERE r.rating < 1\n\n\nCREATE TABLE review (\n  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review\n  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed\n  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review\n  rating FLOAT DEFAULT NULL, -- Rating given in the review\n  text LONGTEXT NOT NULL, -- Content of the review\n  year INTEGER DEFAULT NULL, -- Year the review was posted\n  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted\n  FOREIGN KEY (business_id) REFERENCES business(business_id),\n  FOREIGN KEY (user_id) REFERENCES user(user_id)\n);\n'

In [40]:
question = "list all the businesses which have a review by Niloofar"
generated_sql = generate_query(question)
print(generated_sql)


SELECT b.business_id
FROM business b
WHERE b.business_id IN
    (SELECT b.business_id
     FROM review r
     WHERE r.user_id = 'Niloofar'
       AND r.year = 2023)


In [ ]:
"""
Correct Answer: 

SELECT b.name
FROM review r
JOIN business b ON r.business_id = b.business_id
JOIN user u ON r.user_id = u.user_id
WHERE u.name = "Niloofar";


CREATE TABLE business (
  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business
  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier
  name VARCHAR(255) NOT NULL, -- Name of the business
  full_address VARCHAR(255) NOT NULL, -- Full street address
  city VARCHAR(255) NOT NULL, -- City of the business
  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate
  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate
  review_count BIGINT NOT NULL, -- Number of reviews the business has received
  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)
  rating FLOAT DEFAULT NULL, -- Average rating of the business
  state VARCHAR(255) DEFAULT NULL -- State or province
);

CREATE TABLE review (
  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each review
  business_id VARCHAR(255) NOT NULL, -- ID of the business being reviewed
  user_id VARCHAR(255) NOT NULL, -- ID of the user who wrote the review
  rating FLOAT DEFAULT NULL, -- Rating given in the review
  text LONGTEXT NOT NULL, -- Content of the review
  year INTEGER DEFAULT NULL, -- Year the review was posted
  month VARCHAR(10) DEFAULT NULL, -- Month the review was posted
  FOREIGN KEY (business_id) REFERENCES business(business_id),
  FOREIGN KEY (user_id) REFERENCES user(user_id)
);

CREATE TABLE user (
  uid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each user record
  user_id VARCHAR(255) NOT NULL UNIQUE, -- External user identifier
  name VARCHAR(255) NOT NULL -- Name of the user
);
"""

'\nCorrect Answer: \n\nSELECT b.name\nFROM review r\nJOIN business b ON r.business_id = b.business_id\nJOIN user u ON r.user_id = u.user_id\nWHERE u.name = "Niloofar";\n\n\nCREATE TABLE business (\n  bid INTEGER PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each business\n  business_id VARCHAR(255) NOT NULL UNIQUE, -- External business identifier\n  name VARCHAR(255) NOT NULL, -- Name of the business\n  full_address VARCHAR(255) NOT NULL, -- Full street address\n  city VARCHAR(255) NOT NULL, -- City of the business\n  latitude VARCHAR(255) NOT NULL, -- Latitude coordinate\n  longitude VARCHAR(255) NOT NULL, -- Longitude coordinate\n  review_count BIGINT NOT NULL, -- Number of reviews the business has received\n  is_open TINYINT(1) NOT NULL, -- Indicates if the business is open (1) or closed (0)\n  rating FLOAT DEFAULT NULL, -- Average rating of the business\n  state VARCHAR(255) DEFAULT NULL -- State or province\n);\n\nCREATE TABLE review (\n  rid INTEGER PRIMARY KEY AUTO_INCREMENT, -- 

: 